# data prep road accident dataset       

In [ ]:
import pandas as pd
import numpy as np


from my_modules import my_functions as mybib
from my_modules import project_functions as pr


%load_ext autoreload
%autoreload 2

: 

### Import berlin road accident files and investigate table structure

In [ ]:
accidents18 = pd.read_csv('../data/input/road_accidents/berlin_road_accidents_2018.csv', sep=';', encoding = 'latin1')
print('2018')
display(accidents18.head())

accidents19 = pd.read_csv('../data/input/road_accidents/berlin_road_accidents_2019.csv', sep=';', encoding = 'latin1')
print('2019')
display(accidents19.head())

accidents20 = pd.read_csv('../data/input/road_accidents/berlin_road_accidents_2020.csv', sep=';', encoding = 'latin1')
print('2020')
display(accidents20.head())

accidents21 = pd.read_csv('../data/input/road_accidents/berlin_road_accidents_2021.csv', sep=';', encoding = 'latin1')
print('2021')
display(accidents21.head())

: 

### check columnnames 

In [ ]:
accidents18 = mybib.clean_column_names(accidents18)
accidents19 = mybib.clean_column_names(accidents19)
accidents20 = mybib.clean_column_names(accidents20)
accidents21 = mybib.clean_column_names(accidents21)

#TODO: add a loop for printing
print('2018')
display(accidents18.head())
print(accidents18.shape)
print('2019')
display(accidents19.head())
print(accidents19.shape)
print('2020')
display(accidents20.head())
print(accidents20.shape)
print('2021')
display(accidents21.head())
print(accidents21.shape)

total = len(accidents21) + len(accidents20) + len(accidents19) + len(accidents18)
print(total)


: 

### manually change wrong column names and add missing columns 

In [ ]:
# add empty lor_ab_2021 column on accidents19
#TODO: cleaning function
accidents18.rename(columns={'strzustand':'ustrzustand'}, inplace=True)
accidents18.rename(columns={'istsonstig':'istsonstige'}, inplace=True)
accidents19.insert(5, 'lor_ab_2021', '')
accidents20.insert(4, 'strasse', '')
accidents21.insert(3, 'lor', '')
accidents21.insert(4, 'strasse', '')


columns = len(accidents18.columns)

for i in range(columns):
    print(i)
    print(accidents18.columns[i])
    print(accidents19.columns[i])
    print(accidents20.columns[i])
    print(accidents21.columns[i])

: 

: 

In [ ]:
#check if columnnames are equal

# *dataframes allows to enter multiple dataframes
def column_name_check(*dataframes):
    check = set(dataframes[0].columns)
    
    for df in dataframes[1:]:
        if set(df.columns)!= check:
            return False
    return True 

print(column_name_check(accidents18, accidents19, accidents20, accidents21))

: 

### concat the 4 datasets together  

In [ ]:
accident_data = pd.concat([accidents18, accidents19, accidents20, accidents21], axis=0)
accident_data.shape

accident_data = accident_data.drop(columns=['land'])
display(accident_data.head())
accident_data.dtypes


: 

In [ ]:

accident_data = accident_data.fillna('0')
accident_data = accident_data.replace("",'0')

mybib.nullTable(accident_data)
display(accident_data)


: 

In [ ]:
# changing column istsonstige to int (1 buggy cell insight)
def change_to_int(x):
    if isinstance(x, int):
        return x
    else: 
        if x == '0':
            x = int(0)
        elif x == '1':
            x = int(1)
        else:
            x = int(0)   

        return x 

: 

In [ ]:
accident_data['lor'] = accident_data['lor'].astype(float)
accident_data['lor'] = accident_data['lor'].astype('Int64')

accident_data['lor_ab_2021'] = accident_data['lor_ab_2021'].astype(float)
accident_data['lor_ab_2021'] = accident_data['lor_ab_2021'].astype('Int64')

accident_data['lor_ab_2021'] = accident_data['lor_ab_2021'].astype(float)
accident_data['lor_ab_2021'] = accident_data['lor_ab_2021'].astype('Int64')


accident_data['istsonstige'] = accident_data['istsonstige'].apply(lambda x: change_to_int(x))
accident_data['ustrzustand'] = accident_data['ustrzustand'].apply(lambda x: change_to_int(x))


accident_data['linrefx'] = accident_data['linrefx'].str.replace(",",'.').astype(float)
accident_data['linrefy'] = accident_data['linrefy'].str.replace(",",'.').astype(float)
accident_data['xgcswgs84'] = accident_data['xgcswgs84'].str.replace(",",'.').astype(float)
accident_data['ygcswgs84'] = accident_data['ygcswgs84'].str.replace(",",'.').astype(float)

: 

In [ ]:
accident_data.to_csv('../data/output/raw_accident_dataset_2018-2021.csv', index = False)

display(accident_data.head())
print(accident_data.dtypes)
accident_data.columns




: 

In [ ]:

accident_data = accident_data.drop(columns=['bez'])
print(accident_data.columns)

: 

In [ ]:


accident_data.columns = ['object_id', 'old_lor','street_default', 'lor', 'year', 'month',
       'hour', 'weekday', 'ac_category', 'ac_type', 'ac_type2', 'ac_light',
       'is_bicycle', 'is_car', 'is_pedestrian', 'is_motorcycle', 'is_truck', 'is_other',
       'street_condition', 'linrefx', 'linrefy', 'xgcswgs84', 'ygcswgs84']

print(accident_data.columns)
display(accident_data)

filter = accident_data[accident_data['lor'].isnull()]
filter

key = accident_data['object_id'].astype(str) + '-' + accident_data['year'].astype(str) + '-' + accident_data['lor'].astype(str)

accident_data.insert(0, 'key', key)

display(accident_data)
accident_data.dtypes



: 

: 

In [ ]:
from pyproj import Proj, Transformer
from tqdm import tqdm
# https://de.wikipedia.org/wiki/European_Petroleum_Survey_Group_Geodesy
# https://pyproj4.github.io/pyproj/stable/api/transformer.html
# https://pyproj4.github.io/pyproj/stable/api/proj.html
# https://gis.stackexchange.com/questions/326687/getting-epsg-code-in-pyproj-with-version-2-1-3
# https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
# https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
# https://stackoverflow.com/questions/18603270/progress-indicator-during-pandas-operation.setdefault()
tqdm.pandas()

# ETRS89/UTM Zone 32N
utm = Proj('epsg:25832')

# WGS84
wgs = Proj('epsg:4326')
transformer = Transformer.from_crs('epsg:25832', 'epsg:4326')

lat, lon = transformer.transform(accident_data['linrefx'].tolist(), accident_data['linrefy'].tolist())

print(lat)
print(lon)

accident_data['latitude'] = lat
accident_data['longitude'] = lon

accident_data

: 

In [ ]:
accident_data['latitude']

accident_data.columns

list = ['is_bicycle', 'is_car', 'is_pedestrian', 'is_motorcycle', 'is_truck',
       'is_other']

for l in list:
    accident_data[l] = accident_data[l].astype(bool)
    
    

: 

In [ ]:
accident_data
accident_data.columns

list = ['weekday', 'ac_category', 'ac_type', 'ac_type2', 'ac_light', 'street_condition']

for l in list:
    accident_data[l] = accident_data[l].astype('object')
    
    
accident_data['lor'] = accident_data['lor'].astype(str)
accident_data['old_lor'] = accident_data['old_lor'].astype(str)

def add_zero_at_beginning(x):
    if len(x) == 7:
        return '0' + x
    return x

accident_data['lor'] = accident_data['lor'].apply(add_zero_at_beginning)
accident_data['old_lor'] = accident_data['old_lor'].apply(add_zero_at_beginning)

accident_data

: 

In [ ]:
# accident_data['lor_1'] = accident_data['lor'].str[:-6]
# accident_data['lor_2'] = accident_data['lor'].str[-6:-4]
# accident_data['lor_3'] = accident_data['lor'].str[-4:-2]
# accident_data['lor_4'] = accident_data['lor'].str[-2:]
accident_data['district_LOR3'] = accident_data['lor'].str[:-2]
accident_data['district_LOR2'] = accident_data['lor'].str[:-4]
accident_data['district'] = accident_data['lor'].str[:-6]


: 

: 

In [ ]:
# accident_data['lor_1'].value_counts()

# accident_data['count_num'].value_counts()


# test = accident_data[accident_data['count_num'] == 1]
# test['year'].value_counts()

accident_data

: 

: 

In [ ]:
accident_data.to_csv('../data/output/accident_dataset_2018-2021.csv', index = False)
accident_data.to_excel('../data/output/accident_dataset_2018-2021.xlsx', index = False)
accident_data.to_pickle('../data/output/accident_dataset_2018-2021.pkl')

: 

In [ ]:
#TODO: 2019 füllen

# lor_zuordnung = pd.DataFrame(columns = ['old_lor', 'new_lor'])

# lor_zuordnung['old_lor'] = accident_data['old_lor']
# lor_zuordnung['new_lor'] = accident_data['lor']

# lor_zuordnung 

: 

In [ ]:
lor_zuordnung = lor_zuordnung[lor_zuordnung['old_lor'] > 0]

# lor_zuordnung['new_lor'].astype(int)
# print(lor_zuordnung['old_lor'].value_counts())
# lor_zuordnung['new_lor'].value_counts()



test = lor_zuordnung.drop_duplicates(subset='old_lor')
test['new_lor'] = test['new_lor'].astype(int)
test = test[test['new_lor'] > 0]
test


: 

In [ ]:
def transform_number_under_10(x):
    if int(x) < 10:
        x = '0' + str(int(x))
        return x
    else:
        x = str(int(x))
        return x
    
accident_data['lor_1'] = accident_data['lor_1'].apply(lambda x: transform_number_under_10(x))
accident_data


: 

In [ ]:
import pickle 


with open()

: 

In [ ]:
# accident_data.to_csv('../data/output/accident_dataset_2018-2021.csv', index = False)
# accident_data.to_pickle('../data/output/accident_dataset_2018-2021.pkl')

: 

In [ ]:
import geopandas as gpd 


: 